# Auditoría Sprint 5: Validación de Modelo Final y Adaptadores
**Objetivo:** Verificar que el modelo final (Boosting) se carga correctamente, que el umbral dinámico existe y que el adaptador de datos funciona.



In [ ]:
import os
import sys
import pandas as pd
import json
import pickle
from pycaret.classification import load_model, predict_model

# Añadir directorio raíz para importar src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.adapters import UserInputAdapter # Asumiendo que Jules creó esta clase



## 1. Verificación de Archivos Generados



In [ ]:
model_path = '../models/final_pipeline_v1'
config_path = '../models/model_config.json'

if os.path.exists(model_path + '.pkl'):
    print(f"✅ Modelo encontrado en: {model_path}.pkl")
else:
    print("❌ ERROR: No se encuentra el modelo .pkl")

if os.path.exists(config_path):
    print(f"✅ Configuración encontrada en: {config_path}")
else:
    print("❌ ERROR: No se encuentra model_config.json")



## 2. Carga de Configuración y Umbral Óptimo



In [ ]:
with open(config_path, 'r') as f:
    config = json.load(f)

print("Configuración cargada:")
print(json.dumps(config, indent=2))

threshold = config.get('threshold', 0.5)
print(f"\n🎯 Umbral Óptimo para UI: {threshold}")



## 3. Carga del Modelo



In [ ]:
try:
    # PyCaret carga el pipeline completo
    pipeline = load_model(model_path)
    print("✅ Pipeline cargado exitosamente.")
    print(f"Modelo base: {pipeline.steps[-1][1]}") # Muestra si es XGBoost, LGBM, etc.
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")



## 4. Prueba del Adaptador (User Input -> Model Features)
Simulamos datos que vendrían de Streamlit



In [ ]:
# Datos de prueba "Humanos"
mock_input = {
    "Age": 65,            # Debería convertirse a categoría de edad
    "BMI": 28.5,          # Debería escalarse si es necesario
    "Sex": "Male",        # 1 o 2
    "Smoker": "Yes",      # 1 o 2
    "Diabetes": "Yes",    # 1 o 2
    "PhysicalActivity": "No"
}

print("📥 Input Usuario:", mock_input)

try:
    adapter = UserInputAdapter()
    features_df = adapter.transform(mock_input)

    print("\n📤 Features Transformadas (para el modelo):")
    display(features_df)

    # Validación básica de columnas
    expected_cols = pipeline.feature_names_in_
    missing_cols = set(expected_cols) - set(features_df.columns)

    if len(missing_cols) > 0:
        print(f"⚠️ Advertencia: El adaptador no generó estas columnas (se llenarán con 0 en predicción): {list(missing_cols)[:5]}...")
    else:
        print("✅ Todas las columnas necesarias están presentes.")

except Exception as e:
    print(f"❌ Error en adaptador: {e}")



## 5. Prueba de Inferencia con Probabilidad
Verificamos que el modelo soporte predict_proba para el slider.



In [ ]:
try:
    # Asegurar que el DF tenga todas las columnas (rellenar faltantes con 0 para la prueba)
    full_features = pd.DataFrame(columns=pipeline.feature_names_in_)
    full_features = pd.concat([full_features, features_df], axis=0).fillna(0)

    # 1. Obtener probabilidad cruda
    # Nota: PyCaret encapsula sklearn, a veces es pipeline.predict_proba o predict_model de pycaret
    try:
        proba = pipeline.predict_proba(full_features)[:, 1][0]
        print(f"\n📊 Probabilidad de Riesgo calculada: {proba:.4f} ({proba*100:.2f}%)")
    except:
        print("⚠️ El pipeline directo no respondió a predict_proba, intentando función de PyCaret...")
        predictions = predict_model(pipeline, data=full_features)
        print("Predicciones PyCaret:", predictions)

    # 2. Aplicar Umbral del JSON
    prediction_label = "Alto Riesgo 🔴" if proba >= threshold else "Bajo Riesgo 🟢"
    print(f"DECISIÓN (Umbral {threshold}): {prediction_label}")

except Exception as e:
    print(f"❌ Error en inferencia: {e}")
